In [232]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib qt

In [233]:
# Parameters definition

Nf = 200
fc = 3e9         
df = 1e9                                # Deviation frequency [Hz]
B = (fc+df) - (fc-df)                   # Bandwidth [Hz]
f = np.linspace(fc-df,fc+df,Nf)         # Frequency axis [Hz]
w,wc = 2*np.pi*f, 2*np.pi*fc            # Pulsation axis & central pulsation
sigma_w = 2*np.pi*0.2*df                # Standard deviation of pulsations

A = (1/(np.sqrt(2*np.pi)*sigma_w)) * np.exp(-(w-wc)**2/(2*(sigma_w**2)))        # Amplitude distribution over frequences (Gaussian)

c = 3e8
pi = np.pi
# f = 1.0e10

t = np.linspace(0, 100/f, 100)
exp = np.exp(1j * 2 * pi * f * t)

e0 = 8.85e-12
mu0 = 4*pi*1e-7
l0 = 1.5
n0 = 1
eta0 = 377

n1 = 3
e1 = n1**2 * e0
l1 = 1.0 
eta1 = eta0 / n1

n2 = 1.5
e2 = n2**2 * e0
l2 = 1.0
eta2 = eta0 / n2

n3 = 1
e3 = n3**2 * e0
l3 = 1.5
eta3 = eta0 / n3

# Axis
Nz = 1000           # Precision
z0 = np.linspace(0,l0, int(l0*Nz))
z1 = np.linspace(0,l1, int(l1*Nz))
z2 = np.linspace(0,l2, int(l2*Nz))
z3 = np.linspace(0,l3, int(l3*Nz))

z = np.array([*z0, *z1 + l0, *z2 + (l0+l1), *z3 + (l0+l1+l2)])

In [234]:
def propagation(E0, H0, n, z, eta, freq):
    k = 2 * pi * freq * n / c
    E = E0 * np.cos(k*z) - H0 * 1j * eta * np.sin(k*z)
    H = - E0 * 1j * np.sin(k*z) / eta + H0 * np.cos(k*z)
    return E, H 

In [235]:
# Recursive computation of the reflection coefficients.
# p at the end of variables name indicates primus (value to the right of the interface)

Gamma3 = (n2-n3)/(n2+n3)
Gamma2p = Gamma3 * np.exp(-1j*2*pi*f*n2*l2/c)
rho2 = (n1-n2)/(n1+n2)
Gamma2 = (rho2 + Gamma2p) / (1 + rho2*Gamma2p)
Gamma1p = Gamma2 * np.exp(-1j*2*pi*f*n1*l1/c)
rho1 = (n0-n1)/(n0+n1)
Gamma1 = (rho1 + Gamma1p) / (1 + rho1*Gamma1p)

In [236]:
Ef = 1                                          # E+
Eb = Gamma1                                     # E-

E0int = (Ef + Eb)                               # Actual E field
H0int = (Ef - Eb) / eta0                       # Actual H field

E0 = np.zeros((len(f), len(z0)), dtype=complex); H0 = np.zeros((len(f), len(z0)), dtype=complex)
E1 = np.zeros((len(f), len(z1)), dtype=complex); H1 = np.zeros((len(f), len(z1)), dtype=complex)
E2 = np.zeros((len(f), len(z2)), dtype=complex); H2 = np.zeros((len(f), len(z2)), dtype=complex)
E3 = np.zeros((len(f), len(z3)), dtype=complex); H3 = np.zeros((len(f), len(z3)), dtype=complex)

for i in range(len(f)):

    E0[i], H0[i] = propagation(E0int[i], H0int[i], n0, z0, eta0, f[i]) # -z0?
    E0[i] = np.flip(E0[i])
    H0[i] = np.flip(H0[i])

    E1[i], H1[i] = propagation(E0[i][-1], H0[i][-1], n1, z1, eta1, f[i])

    E2[i], H2[i] = propagation(E1[i][-1], H1[i][-1], n2, z2, eta2, f[i])

    E3[i], H3[i] = propagation(E2[i][-1], H2[i][-1], n3, z3, eta3, f[i])


In [237]:
t = np.linspace(0,1e-6,1000)
Evstime = np.zeros((len(t), len(z)))

for i in range(len(t)):
    for j in range(len(f)):
        Etot = np.concatenate((E0[j], E1[j], E2[j], E3[j]))
        Evstime[i] += A[j]*np.real(Etot*np.exp(1j*w[j]*t[i]))
    
Evstime /= np.max(Evstime)

for elem in Evstime[0]:
    print(elem)


-1.8924233528529235e-06
-1.8844940818997755e-06
-1.8690912769994639e-06
-1.8463103070493348e-06
-1.816276625810192e-06
-1.779145125193482e-06
-1.735099367804974e-06
-1.6843506979506068e-06
-1.6271372362385457e-06
-1.5637227639009231e-06
-1.4943955011973457e-06
-1.4194667832627331e-06
-1.3392696464543773e-06
-1.254157320612181e-06
-1.1645016465178235e-06
-1.0706914171321633e-06
-9.731306537372123e-07
-8.722368218509725e-07
-7.684390003290544e-07
-6.621760085342349e-07
-5.538944966109054e-07
-4.4404701560685414e-07
-3.330900683024819e-07
-2.2148215381098285e-07
-1.0968181395287953e-07
1.85431019770837e-09
1.1267340694080518e-07
2.223283905018927e-07
3.3037976549296467e-07
4.363974482855876e-07
5.399625351648035e-07
6.406690155409358e-07
7.381254134778737e-07
8.319563647913228e-07
9.218041050896852e-07
1.0073298801481446e-06
1.0882152599823981e-06
1.1641633593274415e-06
1.2348999548727902e-06
1.3001744972311102e-06
1.359761011468143e-06
1.4134588898131298e-06
1.4610935617636905e-06
1.5025

In [238]:
# Etot = np.concatenate((E0, E1, E2, E3))
# ztot = np.linspace(-l0, l0+l1+l2+l3, 4500)

# Evstime = np.zeros((len(t), len(Etot)))
# for i in range(len(t)):
#     Evstime[i] = np.real(Etot * exp[i])
    

# plt.plot(ztot, Evstime[0])
# # plt.plot(ztot, Evstime[50])
# plt.show()

In [239]:
# E0 = np.zeros(len(z0))

# for i in t:
#     E0[0] = np.sin(2*pi*f*i)
#     H0[0] = 0
#     E0 = propagation(E0)

In [240]:
# fig, ax = plt.subplots()
# ax.set_xlabel("z [m]")
# ax.set_ylabel("Electric Field [V/m]")
# ax.set_title("Field in layered media")
# ax.grid()
# #ax.set_ylim(-3, 3)

# line, = ax.plot(ztot, Evstime[0], label="Non dispersive medium")
# ax.legend()

# def anim(i):       
#     line.set_ydata(Evstime[i])
#     return line,

# ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=500, repeat=True ,blit=False)
# plt.show()

# # writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
# # ani.save('layered.gif', writer=writer)

In [241]:
c = ["tab:orange", "tab:green", "tab:red", "tab:blue"]

fig, ax = plt.subplots()

ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.grid()
ax.set_ylim(-1, 1)

line, = ax.plot(z, Evstime[0])
yrange = plt.gca().get_ylim()


plt.fill_between(z0, yrange[0], yrange[1], color=c[1],alpha=0.3, label=r"$n=1$")
plt.fill_between(l0+z1, yrange[0], yrange[1], color=c[0],alpha=0.3, label=r"$n=$"+str(n1))
plt.fill_between(l0+l1+z2, yrange[0], yrange[1], color=c[2],alpha=0.3, label=r"$n=$"+str(n2))
plt.fill_between(l0+l1+l2+z3, yrange[0], yrange[1], color=c[1],alpha=0.3)
ax.legend(loc="upper right")

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    line.set_ydata(Evstime[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=200, repeat=True ,blit=False)
plt.show()

#writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
#ani.save('layer_paquet_wave_1f.gif', writer=writer)